In [1]:
import hashlib

from sympy.printing.pytorch import torch

from testing import accuracy
%pylab inline
import numpy as np

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import gzip
import urllib.request
import os

def fetch_mnist(url, filepath):
    """Download and cache MNIST data"""
    if not os.path.exists(filepath):
        print(f"Downloading {url}...")
        urllib.request.urlretrieve(url, filepath)

    with gzip.open(filepath, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16 if 'images' in filepath else 8)

    return data

# Create cache directory
os.makedirs('mnist_data', exist_ok=True)

# Download from official MNIST mirror
base_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

X_train = fetch_mnist(base_url + "train-images-idx3-ubyte.gz",
                      "mnist_data/train-images.gz").reshape(-1, 28, 28)
Y_train = fetch_mnist(base_url + "train-labels-idx1-ubyte.gz",
                      "mnist_data/train-labels.gz")

X_test = fetch_mnist(base_url + "t10k-images-idx3-ubyte.gz",
                     "mnist_data/test-images.gz").reshape(-1, 28, 28)
Y_test = fetch_mnist(base_url + "t10k-labels-idx1-ubyte.gz",
                     "mnist_data/test-labels.gz")

print(f"Training: {X_train.shape}, {Y_train.shape}")
print(f"Test: {X_test.shape}, {Y_test.shape}")

Training: (60000, 28, 28), (60000,)
Test: (10000, 28, 28), (10000,)


In [3]:
X_train.reshape(-1, 28*28).shape

(60000, 784)

In [4]:
import torch
import torch.nn as nn
from tqdm import trange

In [5]:
class GeoNet(torch.nn.Module):
    def __init__(self):
        super(GeoNet, self).__init__()
        self.l1 = nn.Linear(784,128,bias=False)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(128,10,bias=True)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [24]:
model = GeoNet()

# let's train the model
np.random.seed(42)
BS = 32
EPOCHS = 10000
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
for i in (t := trange(EPOCHS)):
    samp = np.random.randint(0, X_train.shape[0], size=(BS))
    X = torch.tensor(X_train[samp].reshape(-1,28*28)).float()
    Y = torch.tensor(Y_train[samp])
    out = model(X)
    # accuracy
    pred = out.argmax(1)
    accuracy = ((pred == Y) / len(Y)).sum()
    optimizer.zero_grad()
    # Loss using categorical entropy loss
    loss = loss_function(out, Y)
    loss.backward()
    optimizer.step()
    t.set_description("loss % .2f accuracy % .2f" % (loss.item(), accuracy.item()))
print("DONE!!")

loss  0.08 accuracy  0.97: 100%|██████████| 10000/10000 [00:16<00:00, 589.70it/s]

DONE!!


In [ ]:
out